In [1]:
from elliot.evaluation.metrics import *

In [21]:
from elliot.evaluation.relevance import Relevance

In [22]:
from types import SimpleNamespace

In [50]:
import pandas as pd
import numpy as np

In [3]:
df_pred = pd.read_csv('./pred.csv')
df_test = pd.read_csv('./test.csv')

In [4]:
df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

In [5]:
df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

In [6]:
df_pred_new.fillna(0, inplace=True)

In [7]:
df_pred_new.loc[df_pred_new.relevance != 0, 'relevance'] = 1

In [8]:
df_pred_new.relevance.value_counts()

0.0    51516
1.0     3144
Name: relevance, dtype: int64

In [11]:
df_pred_new

,user_id,item_id,score,relevance
40,3,590,0.280305,0.0
41,3,1136,0.263707,0.0
42,3,920,0.231234,0.0
43,3,1246,0.188023,0.0
44,3,318,0.187111,0.0
...,...,...,...,...
1179555,71534,1304,0.088851,0.0
1179556,71534,778,0.085875,0.0
1179557,71534,1307,0.082908,0.0
1179558,71534,1258,0.082717,0.0


In [25]:
df_test

,user_id,item_id,rating,timestamp,relevance
0,1256,5378,5.0,2007-10-31 12:18:24,5.0
1,1256,778,4.5,2007-10-31 12:19:51,4.5
2,1256,8376,5.0,2007-10-31 12:30:59,5.0
3,1256,2594,5.0,2007-10-31 12:20:49,5.0
4,1256,30810,4.5,2007-10-31 12:32:24,4.5
...,...,...,...,...,...
38806,68571,112,5.0,2007-01-28 01:37:59,5.0
38807,68571,7256,5.0,2007-01-28 01:36:16,5.0
38808,68571,34405,4.5,2007-01-28 01:13:53,4.5
38809,68571,2618,5.0,2007-01-28 01:43:19,5.0


In [12]:
recommendations = {}

for user, item_id, score in zip(df_pred_new.user_id, df_pred_new.item_id, df_pred_new.score):

    if user in recommendations:
        recommendations[user] += [(item_id, score)]
    else:
        recommendations[user] = [(item_id, score)] 

In [97]:
gt_recommendations = {}

for user, item_id, score in zip(df_pred_new.user_id, df_pred_new.item_id, df_pred_new.relevance):
    
    if score > 0:

        if user in gt_recommendations:
            gt_recommendations[user][item_id] = score
        else:
            gt_recommendations[user] = {item_id: score}

In [13]:
recommendations

{3: [(590, 0.2803048457818345),
  (1136, 0.2637071998923036),
  (920, 0.2312339560954821),
  (1246, 0.18802284081710532),
  (318, 0.1871112119457911),
  (7143, 0.17934610580271013),
  (6377, 0.17279875260521316),
  (3072, 0.17150865359665887),
  (1682, 0.17128326968002658),
  (50, 0.16676174013854336),
  (1610, 0.16613825327357262),
  (4246, 0.1621191534953924),
  (1234, 0.16123816977242386),
  (2355, 0.1603035113260146),
  (2194, 0.15856837000101612),
  (3408, 0.1577896691793699),
  (1073, 0.15500209392500974),
  (4880, 0.15224158350433584),
  (5502, 0.1476469427161324),
  (8529, 0.14385092071067065)],
 51: [(1254, 0.3855216434664601),
  (1247, 0.3788272429633057),
  (2858, 0.3629193540169637),
  (926, 0.3347326798339286),
  (356, 0.33074756782925085),
  (1235, 0.3067464044250825),
  (1267, 0.2932680668182945),
  (4993, 0.28760456680693786),
  (5299, 0.2841238472610524),
  (1213, 0.2328136393607428),
  (1221, 0.23166817464009315),
  (1252, 0.2300372140350201),
  (1219, 0.2222046210665

In [122]:
gt_recommendations

{51: {356: 1.0, 5299: 1.0},
 70: {750: 1.0, 912: 1.0, 1252: 1.0, 1207: 1.0},
 144: {318: 1.0, 589: 1.0, 293: 1.0},
 175: {1201: 1.0, 5995: 1.0},
 190: {6539: 1.0, 1197: 1.0, 1196: 1.0},
 215: {1204: 1.0},
 262: {2959: 1.0, 293: 1.0},
 323: {1250: 1.0, 509: 1.0, 969: 1.0},
 383: {1193: 1.0, 541: 1.0, 924: 1.0},
 463: {50: 1.0},
 465: {1673: 1.0, 1089: 1.0, 6016: 1.0},
 469: {745: 1.0},
 476: {296: 1.0},
 607: {260: 1.0, 50: 1.0, 1213: 1.0, 318: 1.0},
 636: {7153: 1.0},
 657: {1263: 1.0},
 723: {318: 1.0},
 820: {5618: 1.0, 1203: 1.0},
 1008: {594: 1.0},
 1068: {527: 1.0, 7361: 1.0, 3578: 1.0, 2959: 1.0},
 1224: {1220: 1.0},
 1278: {6016: 1.0},
 1297: {37741: 1.0},
 1367: {296: 1.0, 6711: 1.0, 2959: 1.0, 3949: 1.0},
 1391: {7361: 1.0},
 1435: {527: 1.0},
 1466: {778: 1.0, 1258: 1.0, 1136: 1.0, 2762: 1.0},
 1490: {2858: 1.0, 3578: 1.0},
 1499: {1220: 1.0, 1288: 1.0},
 1531: {2918: 1.0},
 1547: {1291: 1.0, 1223: 1.0},
 1590: {6711: 1.0, 2858: 1.0, 4226: 1.0, 318: 1.0},
 1745: {1252: 1.0, 9

In [73]:
Precision(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.057519209659714604

In [71]:
Recall(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.09632131280310642

In [69]:
MAP(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.07298532565140084

In [68]:
MRR(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.18632413351027097

In [67]:
HR(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.47457006952067327

In [66]:
NDCG(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.5),
              cutoff = 20
          )
).eval()

0.08988414852701611

In [123]:
AUC(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.1),
              cutoff = 20,
              num_items=20,
              data=SimpleNamespace(train_dict={x:[] for x in recommendations.keys()})
          )
).eval()

0.626898329176478

In [110]:
GAUC(recommendations, None, None, 
          eval_objects = SimpleNamespace(
              relevance=Relevance(gt_recommendations, 0.1),
              cutoff = 20,
              num_items=20,
              data=SimpleNamespace(train_dict={x:[] for x in recommendations.keys()})
          )
).eval()

0.6187365226601701

In [130]:
LAUC(
    recommendations, 
    SimpleNamespace(config_test=False), 
    None, 
    eval_objects = SimpleNamespace(
      relevance=Relevance(gt_recommendations, 0.1),
      cutoff = 20,
      num_items=20,
      data=SimpleNamespace(train_dict={x:[] for x in recommendations.keys()})
    )
).eval()

0.6187365226601701